# **Feature Extraction Methods: Balanced Data With Annotations and Augmentation**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, RMS Power]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression), Filtering]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random
import IPython.display as ipd

In [13]:
# Variabels to be reused
path =        'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/audio_files' 
npy_path =    'C:/Users/thato/Documents/Final-Year-Project/Dataset/test_audio_npy/' 
test_csv =    'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test-annotated.csv' 
sr = 22050

In [14]:
test_data = pd.read_csv(test_csv)

In [15]:
test_data['species'].value_counts()

species
Ixobrychus minutus           185
Botaurus stellaris           155
Charadrius alexandrinus      130
Dendrocopos minor            116
Acrocephalus arundinaceus    114
Circus aeruginosus           107
Fulica atra                   96
Coracias garrulus             88
Himantopus himantopus         81
Motacilla flava               74
Tachybaptus ruficollis        64
Gallinula chloropus           46
Alcedo atthis                 44
Ardea purpurea                43
Acrocephalus melanopogon      42
Anas platyrhynchos            32
Acrocephalus scirpaceus       31
Ciconia ciconia               25
Porphyrio porphyrio           25
Anas strepera                 20
Name: count, dtype: int64

In [16]:
test_data.head()

,Unnamed: 0,species,audio_name,duration,filename_npy,start,end,label
0,0,Acrocephalus scirpaceus,XC421435.mp3,9.247347,Acrocephalus scirpaceus/XC421435.npy,0.063410,7.947447,song
1,1,Tachybaptus ruficollis,XC459394.mp3,7.784490,Tachybaptus ruficollis/XC459394.npy,1.556898,5.284557,song
2,2,Fulica atra,XC543771.mp3,3.059002,Fulica atra/XC543771.npy,1.013850,1.999042,call
3,3,Coracias garrulus,XC318427.mp3,760.058776,Coracias garrulus/XC318427.npy,98.699061,99.241941,call
4,4,Coracias garrulus,XC318427.mp3,760.058776,Coracias garrulus/XC318427.npy,103.490146,104.738831,call


# **Creating a class to do the extraction**

In [9]:
class Extraction:
  def __init__(self, test_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=128, n_mfcc=20, n_chroma=12, n_cqt=84, hoplength=256, features=['mfcc'], normalize=True, avgpool=False, augment=False):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      n_cqt (int): Number of CQT bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'cqt', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      avgpool (bool): Whether to avgpool the features.
    """
    
    self.test_df = test_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma
    self.n_cqt = n_cqt
    self.hoplength = hoplength
    
     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'cqt', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'cqt', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool
    self.augment = augment

    print(f"Test DataFrame shape: {test_df.shape}")

    # extract train and val labels and features
    self.test_y, self.test_features, self.test_ids = self.feature_extraction(self.test_df, window_size=self.window_size)

    # process the features by average pooling
    self.test_features_2D, self.test_features_1D,  = self.process_features(self.test_features)

  def random_augmentation(self, audio):
        """
        Apply random augmentation to the audio: pitch shift, time stretch, or add noise.
        """
        aug_type = np.random.choice(['pitch_up', 'pitch_down', 'noise', 'stretch', 'shrink'])

        if aug_type == 'pitch_up':
            # Random pitch up (between 1 and 4 semitones)
            pitch_shift = np.random.uniform(1, 4)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'pitch_down':
            # Random pitch down (between -1 and -4 semitones)
            pitch_shift = np.random.uniform(-4, -1)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'noise':
            # Add random noise at a fraction of the original audio volume
            noise = self.generate_pink_noise(len(audio))
            audio = audio + 0.01 * np.random.uniform(0.2, 0.5) * np.max(audio) * noise
        elif aug_type == 'stretch':
            # Randomly stretch audio (speed up)
            stretch_factor = np.random.uniform(1.1, 1.5)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)
        elif aug_type == 'shrink':
            # Randomly shrink audio (slow down)
            stretch_factor = np.random.uniform(0.8, 0.9)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)

        return audio

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio
  
  def avg_pooling_keras(self, feature):
    # Clear the previous Keras session
    tf.keras.backend.clear_session()

    # Define the input shape based on features
    input_shape = feature.shape[1:]  # (n_mels, time_steps)

    # Create the Keras model for average pooling
    inputs = tf.keras.layers.Input(shape=input_shape)
    pooled = tf.keras.layers.GlobalAveragePooling1D()(inputs)
    pooling_model = tf.keras.models.Model(inputs=inputs, outputs=pooled)

    # Perform pooling using the model
    pooled_features = pooling_model.predict(feature)

    return pooled_features

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_cqt(self, window):
    cqt = librosa.cqt(y=window, sr=sr, hop_length=self.hoplength, n_bins=self.n_cqt)
    cqt_db = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)
    return cqt_db

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels, hop_length=self.hoplength)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if self.normalize:
      return librosa.util.normalize(mel_db)
    else:
      return mel_db
    
  def avgpooling(self, train_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)

    return train_X

    
  def process_features(self, train_features_dict):
    train_copy = train_features_dict.copy()
    for each in train_copy.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'cqt':
        n_features=self.n_cqt
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_copy[each]

      if self.avgpool:
        train_copy[each] = self.avgpooling(train_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_copy[each] = train_copy[each]
    
    return train_features_dict, train_copy
      

  def feature_extraction(self, dataframe, window_size):
    y = [] # To hold the labels
    ids = []
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path =  os.path.join(self.npy_path, row['filename_npy'])
          id = row['audio_name']
          start = row['start']
          end = row['end']

          # print(f"Processing file: {file_path}")

          try:
            
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          start = int(start * sr)
          end = int(end * sr)+512



          if end > len(audio):
             end = len(audio)

          sample = audio[start:end]

          if len(sample) < 512:
                continue

          sample = self.normalize_audio(sample)

          sample = self.pad_with_noise(sample, window_length=self.window_size, window_samples=len(sample))
          

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(sample, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)
              ids.append(id)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)
    ids = np.array(ids)

          # If not using average pooling, return resized features
    return y, features_dict, ids

## **Window Size = 3s**

In [7]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [17]:
features = Extraction(test_data,
                      window_size=3,
                      features=features_list,
                      avgpool=True
                      )

Test DataFrame shape: (1518, 8)
Number of rows in dataframe: 1518


Processing data:   0%|          | 0/1518 [00:00<?, ?it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 1518/1518 [03:06<00:00,  8.14it/s]



76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step


In [18]:
test_y = features.test_y

display(test_y.shape)

(2426,)

In [19]:
# Avgpooled Features
test_features_1D = features.test_features_1D
for key in test_features_1D.keys():
  display(key)
  display(test_features_1D[key].shape)

'melspectrogram'

(2426, 128)

'mfcc'

(2426, 20)

'chroma'

(2426, 12)

'cqt'

(2426, 84)

In [20]:
# Not avgpooled Features
test_features_2D = features.test_features_2D
for key in test_features_2D.keys():
  display(key)
  display(test_features_2D[key].shape)

'melspectrogram'

(2426, 128, 259)

'mfcc'

(2426, 20, 259)

'chroma'

(2426, 12, 259)

'cqt'

(2426, 84, 259)

In [21]:
test_ids = features.test_ids

### Encode Classes

In [22]:
label_encoder = LabelEncoder().fit(test_y)
test_y_encoded = label_encoder.transform(test_y)

classes = list(label_encoder.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]))
print("Encoded classes:", classes)
print("Encoded training labels:", test_y_encoded)

Encoded classes: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus', 'Alcedo atthis', 'Anas platyrhynchos', 'Anas strepera', 'Ardea purpurea', 'Botaurus stellaris', 'Charadrius alexandrinus', 'Ciconia ciconia', 'Circus aeruginosus', 'Coracias garrulus', 'Dendrocopos minor', 'Fulica atra', 'Gallinula chloropus', 'Himantopus himantopus', 'Ixobrychus minutus', 'Motacilla flava', 'Porphyrio porphyrio', 'Tachybaptus ruficollis']
Encoded training labels: [2 2 2 ... 3 3 3]


In [23]:
# Avg Pooled

test_features_1D['label'] = test_y_encoded


test_features_1D['id'] = test_ids


# Not Avg Pooled

test_features_2D['label'] = test_y_encoded

test_features_2D['id'] = test_ids


In [24]:
merged_dict_1D = {'test': test_features_1D}
merged_dict_1D

{'test': {'melspectrogram': array([[-0.01765514, -0.97839206, -0.9803045 , ..., -0.7142255 ,
          -0.7470485 , -0.87501496],
         [-0.01766029, -0.9785224 , -0.9804503 , ..., -0.71047103,
          -0.74089056, -0.87236935],
         [-0.01765937, -0.97845495, -0.9803674 , ..., -0.71974105,
          -0.7478806 , -0.8759612 ],
         ...,
         [-0.06101144, -0.47259116, -0.5047452 , ..., -0.9487564 ,
          -0.9546229 , -0.99134564],
         [-0.08638648, -0.4008829 , -0.44541013, ..., -0.97197163,
          -0.9778899 , -0.9878369 ],
         [-0.09473609, -0.37721655, -0.425733  , ..., -0.96823037,
          -0.97875863, -0.9913111 ]], dtype=float32),
  'mfcc': array([[-1.        , -0.27609986, -0.13612537, ...,  0.0307789 ,
           0.02261043,  0.0424169 ],
         [-1.        , -0.31020635, -0.1558111 , ...,  0.03040246,
           0.03435915,  0.04467259],
         [-0.99837524, -0.29583684, -0.13935724, ...,  0.03293548,
           0.03335591,  0.04711575],

In [25]:
merged_dict_2D = {'test': test_features_2D}
merged_dict_2D

{'test': {'melspectrogram': array([[[-0.01723288,  0.        , -0.00813701, ..., -0.01359067,
           -0.00139572, -0.00619582],
          [-0.19237295, -0.2118108 , -0.27007061, ..., -0.31833053,
           -0.238718  , -0.19343433],
          [-0.26630595, -0.28599623, -0.3431657 , ..., -0.39065692,
           -0.31489858, -0.26669401],
          ...,
          [-0.98350704, -0.97421175, -0.94003344, ..., -0.92514271,
           -0.97606236, -0.94283426],
          [-0.99333322, -0.99142528, -0.96992719, ..., -0.94783878,
           -0.98686194, -0.94328469],
          [-0.99557692, -1.        , -1.        , ..., -1.        ,
           -1.        , -0.94768447]],
  
         [[-0.01728454,  0.        , -0.00815093, ..., -0.01361341,
           -0.00138896, -0.00640995],
          [-0.19296663, -0.21164858, -0.2705158 , ..., -0.31889537,
           -0.23782443, -0.19997455],
          [-0.26712748, -0.28577632, -0.34373018, ..., -0.39136094,
           -0.31372395, -0.27571216],
 

### Save the merged dictionary to a pkl

In [27]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/AnnotatedTest/test_features_3s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [28]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated Test/test_features_3s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file